# Chapter 7

## Raster Data

In [ ]:
# Downloading the data, if not yet downloaded

import wget
import os

if not os.path.exists("data.zip"):
    url = 'http://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip'
    wget.download(url,bar=None)
else:
    print("ok. data already downloaded.")

In [ ]:
# unzip the data if not yet unzipped

import zipfile

if not os.path.exists("landsat"):
    zip_ref = zipfile.ZipFile("data.zip", 'r')
    zip_ref.extractall("landsat")
    zip_ref.close()
else:
    print("ok. data already unzipped.")

In [ ]:
# Now we have the tif file "landsat/LandsatMos25.tif" extracted

import rasterio

ds = rasterio.open('landsat/LandsatMos25.tif', 'r')

In [ ]:
print(ds.name)
print(ds.count) # number of raster bands, in our case 3 for r,g,b
print(ds.width, ds.height)

In [ ]:
ds.dtypes

In [ ]:
print(ds.crs)

In [ ]:
print(ds.bounds)

In [ ]:
print(ds.transform)  # affine transformation pixel to crs+

In [ ]:
from rasterio.crs import CRS

ds = rasterio.open('landsat/LandsatMos25.tif', 'r+')
crs = CRS.from_epsg(21781)
ds.crs = crs
print(ds.crs)

In [ ]:
ds.transform * (0, 0)    # Pixel to CRS

In [ ]:
~ds.transform # inverse affine transformation
~ds.transform * (0,0) # CRS to Pixel
px, py = ~ds.transform * (612073, 267991)
print(px,py)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
r = ds.read(1)
g = ds.read(2)
b = ds.read(3)

In [ ]:
rgb = np.dstack((r,g,b))  # stack r,g,b so we can display it...

In [ ]:
fig, ax = plt.subplots(figsize=(15,9))
ax.imshow(rgb, interpolation='bilinear')
ax.plot(px,py, 'ro'); 

## Vector Data

We're downloading earthquake data from USGS:
https://earthquake.usgs.gov/earthquakes/feed/v1.0/geojson.php

This data is updated every minute

* 2.5_week.geojson: All earthquakes > 2.5 from the last week
* 2.5_month.geojson: All earthquakes with a magnitude > 2.5 from the last month

We're looking at the data from last week

In [ ]:
import requests

url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_week.geojson"
#url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_month.geojson"

data = requests.get(url)
file = open("earthquakes.geojson","wb")
file.write(data.content)
file.close()

Now let's import geopandas

In [ ]:
import geopandas as gpd

eq_gdf = gpd.read_file("earthquakes.geojson")
eq_gdf.head()

Now we use GeoPandas to load the dataset

Let's simplify the output and only take most important rows

In [ ]:
eq = eq_gdf[["time","mag", "place","geometry"]].copy()
eq.head()

Look at the histogramm:

In [ ]:
%matplotlib inline
eq.mag.hist(bins=16);

Timestamps in UTC are not really human readable...
Let's convert them

In [ ]:
from datetime import datetime, timezone

data = []
for row in range(0,len(eq)):
    time = eq.iloc[row].time
    t = str(datetime.fromtimestamp(time/1000.0, timezone.utc))
    data.append(t)
    
eq["time_utc"] = data
eq.head()

In [ ]:
eq = eq.drop(['time'], axis=1)

In [ ]:
eq.plot();

Open Natural Earth Dataset with all Polygons of all countries

In [ ]:
gdfAdmin0 = gpd.read_file("data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp", encoding="utf-8")
gdfAdmin0.head()

In [ ]:
countries = gdfAdmin0.plot(figsize=(15,9), color="black")


In [ ]:
eq.sort_values(["mag"], ascending=False).head()

Point Clouds


first we unzip the folder to get the .las file

In [7]:
import zipfile
import os


if not os.path.exists("../data/points/26825_12475.las"):
    zip_ref = zipfile.ZipFile("../data/points/26825_12475.zip", 'r')
    zip_ref.extractall("../data/points/")
    zip_ref.close()
else:
    print("ok. data already unzipped.")

In [ ]:
from laspy.file import File
import numpy as np

file = File('../data/points/26825_12475.las', mode='r')
coords = np.dstack((file.x, file.y, file.z))
file.close()
print(coords)